## Rozwiązywany jest problem wyznaczenia minimalnej mocy jaka ma wydzielić sie na układzie dostarczonym na zajeciach (jest to moc strat cieplnych - chcemy minimalizować straty)
### rozwiązywany jest on jako problem liniowy za pomoca biblioteki pulp
  
zmiennymi liniowymi sa wartości odwrotności rezystancji ukśadu, taka konwencja (zamiast wykorzystania bezpośrednio rezystancji) uprosciła zapis
ograniczeniami są rownania pradowo-napieciowe, ktore muszą być spełnione
problem jest typu miminalizującego
funkcją zysku (tutaj ograniczenia straty) jest całkowita moc wydzielana na układzie, czyli odpowiednio przemnożone i dodane zmienne liniowe

![Zdj układu](./images/mostek.png)

### znaczenie wiekszości wywoływanych funkcji i tworzonych zmiennych opisana jest komentarzami w programie

In [1]:
import pulp
print(pulp.__version__)

2.3.1


In [2]:
from pulp import *

In [15]:
prob = LpProblem("MostekRezystorow", LpMinimize)
#zmiennymi sa odwrotnosci rezystancji, bo tak latwiej zlinearyzowac
x1 = LpVariable("R1",0.0001)
x2 = LpVariable("R2",0.0001)
x3 = LpVariable("R3",0.0001)
x4 = LpVariable("R4",0.0001)
x5 = LpVariable("R5",0.0001)

#problem minimalizacji mocy U^2 / R, gdzie U/R musi sie miescic w zakresie pradu
prob += 36 * x1 + 100 * x2 + 16 * x3 + 49 * x4 + 9 * x5, "Calkowita moc"
#ograniczenia   I - delta < U/R < I + delta
prob += 6 * x1 <= 5e-3
prob += 6 * x1 >= 3e-3
prob += 10 * x2 <= 3e-3
prob += 10 * x2 >= 1e-3
prob += 4 * x3 <= 3e-3
prob += 4 * x3 >= 1e-3
prob += 7 * x4 <= 3e-3
prob += 7 * x4 >= 1e-3
prob += 3 * x5 <= 5e-3
prob += 3 * x5 >= 3e-3


prob.writeLP("MostekRezystorow.lp") #zapis do pliku
prob.solve() #rozwiazanie
print("Status:", LpStatus[prob.status]) #status rozwiazania

for v in prob.variables(): #obliczone wartosci zmiennych
    print(v.name, "=", (1/v.varValue)) #odwrotnosc bo rezystancje
print("Calkowita moc = ", value(prob.objective))



Status: Optimal
R1 = 2000.0
R2 = 10000.0
R3 = 4000.0
R4 = 7000.000140000004
R5 = 1000.0
Calkowita moc =  0.04799999986


 #### Poprzez odwrócenie zwracanych wartości otrzymujemy poszukiwane w zadaniu rezystancji (mimo w pulpie używania ich odwrotności)
 Otrzymana moc jest niewielka, można zatem uznać problem za rozwiązany, według PuLP jest to rozwiązanie optymalne